# Amazon Nova Act SDK를 사용한 기본 Browser tool 사용법

## 개요

이 튜토리얼에서는 Nova Act SDK를 Amazon Bedrock Agentcore Browser tool과 함께 사용하는 방법을 배웁니다. browser tool을 headless로 사용하고 브라우저를 실시간으로 보는 예제를 제공합니다.


### 튜토리얼 세부사항


| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형                                                                            |
| Agent 유형          | 단일                                                                              |
| Agentic Framework   | Nova Act                                                                         |
| LLM model           | Amazon Nova Act model                                                            |
| 튜토리얼 구성요소   | NovaAct를 사용하여 headless 방식으로 browser tool과 상호작용                     |
| 튜토리얼 분야       | vertical                                                                         |
| 예제 복잡도         | 쉬움                                                                              |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK, Nova Act                                     |

### 튜토리얼 아키텍처

이 튜토리얼에서는 Nova Act를 browser tool과 함께 사용하는 방법을 설명합니다.  

예제에서는 Nova Act agent에 자연어 지시사항을 전송하여 Bedrock Agentcore 브라우저에서 headless 방식으로 작업을 수행합니다.

### 튜토리얼 주요 기능

* headless 방식으로 browser tool 사용
* Nova Act를 browser tool과 함께 사용

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명. IAM role/user는 다음 권한이 있어야 합니다 https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-onboarding.html#browser-credentials-config
* Amazon Bedrock AgentCore SDK
* Nova Act SDK 및 API key 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Bedrock Agentcore Browser tool과 함께 NovaAct 사용하기
로컬에서 실행할 수 있는 Python 스크립트를 생성합니다. 스크립트에서 Nova Act는 브라우저 세션의 CDP endpoint를 사용하여 연결하고 playwright actuations를 수행합니다. 

In [ ]:
%%writefile basic_browser_with_nova_act.py
"""Browser automation script using Amazon Bedrock AgentCore and Nova Act.

This script demonstrates AI-powered web automation by:
- Initializing a browser session through Amazon Bedrock AgentCore
- Connecting to Nova Act for natural language web interactions
- Performing automated searches and data extraction using browser
"""

from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
from rich.console import Console
import argparse
import json

console = Console()

from boto3.session import Session

# boto3 세션에서 현재 region 가져오기
boto_session = Session()
region = boto_session.region_name
print("using region", region)

def browser_with_nova_act(prompt, starting_page, nova_act_key, region="us-west-2"):
    result = None
    # browser_session context manager로 브라우저 세션 생성 및 자동 정리
    with browser_session(region) as client:
        # CDP(Chrome DevTools Protocol) WebSocket URL과 인증 헤더 생성
        ws_url, headers = client.generate_ws_headers()
        try:
            # NovaAct를 CDP endpoint에 연결하여 초기화
            with NovaAct(
                cdp_endpoint_url=ws_url,  # Bedrock AgentCore 브라우저의 CDP endpoint
                cdp_headers=headers,  # 인증을 위한 헤더
                preview={"playwright_actuation": True},  # Playwright 기반 브라우저 제어 활성화
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                # 자연어 prompt를 브라우저 액션으로 변환하여 실행
                result = nova_act.act(prompt)
        except Exception as e:
            console.print(f"NovaAct error: {e}")
        finally:
            return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="Browser Search instruction")
    parser.add_argument("--starting-page", required=True, help="Starting URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API key")
    parser.add_argument("--region", default="us-west-2", help="AWS region")
    args = parser.parse_args()

    result = browser_with_nova_act(
        args.prompt, args.starting_page, args.nova_act_key, args.region
    )
    console.print(f"\n[cyan] Response[/cyan] {result.response}")
    console.print(f"\n[bold green]Nova Act Result:[/bold green] {result}")

#### 스크립트 실행하기
스크립트를 실행하기 전에 아래에 Nova Act API key를 붙여넣으세요. 

In [ ]:
NOVA_ACT_KEY= '' ### 여기에 Nova Act Key를 붙여넣으세요

In [ ]:
!python basic_browser_with_nova_act.py --prompt "Search for macbooks and extract the details of the first one" --starting-page "https://www.amazon.com/" --nova-act-key {NOVA_ACT_KEY}

In [ ]:
!python basic_browser_with_nova_act.py --prompt "Extract and return Amazon revenue for the last 4 years" --starting-page "https://stockanalysis.com/stocks/amzn/financials/" --nova-act-key {NOVA_ACT_KEY}

## 내부적으로 무슨 일이 일어났나요?

* `browser_session`을 사용하여 browser client를 인스턴스화하면, Browser client가 생성되고 세션이 시작됩니다
* 그런 다음 `cdp_endpoint_url`과 `cdp_headers`를 사용하여 해당 브라우저 세션을 가리키도록 Nova Act를 구성했습니다 
* 이제 Nova ACT SDK가 자연어 지시사항을 받아 브라우저에서 Playwright actuations를 생성했습니다. 

# 축하합니다!